# Homework 2

## FINM 35700 - Spring 2024

### UChicago Financial Mathematics

* Alex Popovici
* alex.popovici@uchicago.edu

This homework relies on:

- the corporate and government bonds symbology file `bond_symbology`, 
- the "on-the-run" treasuries data file `govt_on_the_run`,
- the bond market data file `bond_market_prices_eod`, containing EOD price data as of 2024-04-08.

# Problem 1: Constructing fixed rate bonds

In [49]:
import QuantLib as ql
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

# Use static calculation/valuation date of 2024-04-08, matching data available in the market prices EOD file
calc_date = ql.Date(8, 4, 2024)
ql.Settings.instance().evaluationDate = calc_date
date=pd.to_datetime('2024-04-08')



## a. Prepare the symbology and market data files for fixed rate government and corporate bonds

Load the `bond_symbology`, `bond_market_prices_eod` and `govt_on_the_run` Excel files into dataframes. Filter the symbology frame for fixed rate bonds only (cpn_type="FIXED").


In [50]:
import pandas as pd

bond_symbology = pd.read_excel('/Users/rogerlin/Desktop/Temporary/UChicago_FINM_35700_CreditMarkets_Spring2024_Homework_2 2/data/bond_symbology.xlsx')
bond_market_prices_eod = pd.read_excel('/Users/rogerlin/Desktop/Temporary/UChicago_FINM_35700_CreditMarkets_Spring2024_Homework_2 2/data/bond_market_prices_eod.xlsx')
govt_on_the_run = pd.read_excel('/Users/rogerlin/Desktop/Temporary/UChicago_FINM_35700_CreditMarkets_Spring2024_Homework_2 2/data/govt_on_the_run.xlsx')

bond_symbology['term'] = (bond_symbology['maturity'] - bond_symbology['start_date']).dt.days / 365.25
bond_symbology['TTM'] = (bond_symbology['maturity'] - date).dt.days / 365.25
bond_market_prices_eod['midPrice'] = 0.5*(bond_market_prices_eod['bidPrice'] + bond_market_prices_eod['askPrice'])
bond_market_prices_eod['midYield'] = 0.5*(bond_market_prices_eod['bidYield'] + bond_market_prices_eod['askYield'])

fixed_rate_bonds_df = bond_symbology[bond_symbology['cpn_type'] == "FIXED"]
fixed_rate_bonds_df

,ticker,class,figi,isin,und_bench_isin,security,name,type,coupon,cpn_type,...,acc_first,maturity,mty_typ,rank,amt_out,country,currency,status,term,TTM
0,AAPL,Corp,BBG004HST0K7,US037833AL42,US912810TZ12,AAPL 3.85 05/04/43,APPLE INC,GLOBAL,3.850,FIXED,...,2013-05-03,2043-05-04,AT MATURITY,Sr Unsecured,3000.000,US,USD,ACTV,30.001369,19.069131
1,AAPL,Corp,BBG006F8VWJ7,US037833AT77,US912810TZ12,AAPL 4.45 05/06/44,APPLE INC,GLOBAL,4.450,FIXED,...,2014-05-06,2044-05-06,AT MATURITY,Sr Unsecured,1000.000,US,USD,ACTV,30.001369,20.076660
2,AAPL,Corp,BBG0081TNL50,US037833BA77,US912810TZ12,AAPL 3.45 02/09/45,APPLE INC,GLOBAL,3.450,FIXED,...,2015-02-09,2045-02-09,AT MATURITY,Sr Unsecured,2000.000,US,USD,ACTV,30.001369,20.840520
3,AAPL,Corp,BBG008N1BQC1,US037833BH21,US912810TZ12,AAPL 4 3/8 05/13/45,APPLE INC,GLOBAL,4.375,FIXED,...,2015-05-13,2045-05-13,AT MATURITY,Sr Unsecured,2000.000,US,USD,ACTV,30.001369,21.095140
4,AAPL,Corp,BBG00C7QBCQ1,US037833BW97,US91282CJZ59,AAPL 4 1/2 02/23/36,APPLE INC,GLOBAL,4.500,FIXED,...,2016-02-23,2036-02-23,CALLABLE,Sr Unsecured,1250.000,US,USD,ACTV,20.000000,11.876797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,DIS,Corp,BBG00N3BD9G0,USU25497AR66,US91282CKG59,DIS 7 1/8 04/08/28,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.125,FIXED,...,2019-03-20,2028-04-08,AT MATURITY,Sr Unsecured,10.285,US,USD,ACTV,9.054073,4.000000
820,DIS,Corp,BBG00N3BZ921,USU25497AW51,US91282CJZ59,DIS 8.45 08/01/34,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.450,FIXED,...,2019-03-20,2034-08-01,AT MATURITY,Sr Unsecured,1.000,US,USD,ACTV,15.367556,10.313484
821,DIS,Corp,BBG00N55NLL7,USU25497BM60,US912810TV08,DIS 7.9 12/01/2095,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.900,FIXED,...,2019-03-20,2095-12-01,AT MATURITY,Sr Unsecured,0.502,US,USD,ACTV,76.700890,71.646817
822,DIS,Corp,BBG00N55NPV7,USU25497BN44,US912810TV08,DIS 8 1/4 10/17/2096,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.250,FIXED,...,2019-03-20,2096-10-17,AT MATURITY,Sr Unsecured,0.158,US,USD,ACTV,77.579740,72.525667


In [51]:
bond_symbology['term'] = (bond_symbology['maturity'] - bond_symbology['start_date']).dt.days / 365.25
bond_symbology['TTM'] = (bond_symbology['maturity'] - date).dt.days / 365.25
bond_market_prices_eod['midPrice'] = 0.5*(bond_market_prices_eod['bidPrice'] + bond_market_prices_eod['askPrice'])
bond_market_prices_eod['midYield'] = 0.5*(bond_market_prices_eod['bidYield'] + bond_market_prices_eod['askYield'])

fixed_rate_bonds_df = bond_symbology[bond_symbology['cpn_type'] == "FIXED"]
fixed_rate_bonds_df

,ticker,class,figi,isin,und_bench_isin,security,name,type,coupon,cpn_type,...,acc_first,maturity,mty_typ,rank,amt_out,country,currency,status,term,TTM
0,AAPL,Corp,BBG004HST0K7,US037833AL42,US912810TZ12,AAPL 3.85 05/04/43,APPLE INC,GLOBAL,3.850,FIXED,...,2013-05-03,2043-05-04,AT MATURITY,Sr Unsecured,3000.000,US,USD,ACTV,30.001369,19.069131
1,AAPL,Corp,BBG006F8VWJ7,US037833AT77,US912810TZ12,AAPL 4.45 05/06/44,APPLE INC,GLOBAL,4.450,FIXED,...,2014-05-06,2044-05-06,AT MATURITY,Sr Unsecured,1000.000,US,USD,ACTV,30.001369,20.076660
2,AAPL,Corp,BBG0081TNL50,US037833BA77,US912810TZ12,AAPL 3.45 02/09/45,APPLE INC,GLOBAL,3.450,FIXED,...,2015-02-09,2045-02-09,AT MATURITY,Sr Unsecured,2000.000,US,USD,ACTV,30.001369,20.840520
3,AAPL,Corp,BBG008N1BQC1,US037833BH21,US912810TZ12,AAPL 4 3/8 05/13/45,APPLE INC,GLOBAL,4.375,FIXED,...,2015-05-13,2045-05-13,AT MATURITY,Sr Unsecured,2000.000,US,USD,ACTV,30.001369,21.095140
4,AAPL,Corp,BBG00C7QBCQ1,US037833BW97,US91282CJZ59,AAPL 4 1/2 02/23/36,APPLE INC,GLOBAL,4.500,FIXED,...,2016-02-23,2036-02-23,CALLABLE,Sr Unsecured,1250.000,US,USD,ACTV,20.000000,11.876797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,DIS,Corp,BBG00N3BD9G0,USU25497AR66,US91282CKG59,DIS 7 1/8 04/08/28,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.125,FIXED,...,2019-03-20,2028-04-08,AT MATURITY,Sr Unsecured,10.285,US,USD,ACTV,9.054073,4.000000
820,DIS,Corp,BBG00N3BZ921,USU25497AW51,US91282CJZ59,DIS 8.45 08/01/34,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.450,FIXED,...,2019-03-20,2034-08-01,AT MATURITY,Sr Unsecured,1.000,US,USD,ACTV,15.367556,10.313484
821,DIS,Corp,BBG00N55NLL7,USU25497BM60,US912810TV08,DIS 7.9 12/01/2095,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.900,FIXED,...,2019-03-20,2095-12-01,AT MATURITY,Sr Unsecured,0.502,US,USD,ACTV,76.700890,71.646817
822,DIS,Corp,BBG00N55NPV7,USU25497BN44,US912810TV08,DIS 8 1/4 10/17/2096,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.250,FIXED,...,2019-03-20,2096-10-17,AT MATURITY,Sr Unsecured,0.158,US,USD,ACTV,77.579740,72.525667


In [52]:
fixed_rate_bonds_df = bond_symbology[bond_symbology['cpn_type'] == "FIXED"]
fixed_rate_bonds_df

,ticker,class,figi,isin,und_bench_isin,security,name,type,coupon,cpn_type,...,acc_first,maturity,mty_typ,rank,amt_out,country,currency,status,term,TTM
0,AAPL,Corp,BBG004HST0K7,US037833AL42,US912810TZ12,AAPL 3.85 05/04/43,APPLE INC,GLOBAL,3.850,FIXED,...,2013-05-03,2043-05-04,AT MATURITY,Sr Unsecured,3000.000,US,USD,ACTV,30.001369,19.069131
1,AAPL,Corp,BBG006F8VWJ7,US037833AT77,US912810TZ12,AAPL 4.45 05/06/44,APPLE INC,GLOBAL,4.450,FIXED,...,2014-05-06,2044-05-06,AT MATURITY,Sr Unsecured,1000.000,US,USD,ACTV,30.001369,20.076660
2,AAPL,Corp,BBG0081TNL50,US037833BA77,US912810TZ12,AAPL 3.45 02/09/45,APPLE INC,GLOBAL,3.450,FIXED,...,2015-02-09,2045-02-09,AT MATURITY,Sr Unsecured,2000.000,US,USD,ACTV,30.001369,20.840520
3,AAPL,Corp,BBG008N1BQC1,US037833BH21,US912810TZ12,AAPL 4 3/8 05/13/45,APPLE INC,GLOBAL,4.375,FIXED,...,2015-05-13,2045-05-13,AT MATURITY,Sr Unsecured,2000.000,US,USD,ACTV,30.001369,21.095140
4,AAPL,Corp,BBG00C7QBCQ1,US037833BW97,US91282CJZ59,AAPL 4 1/2 02/23/36,APPLE INC,GLOBAL,4.500,FIXED,...,2016-02-23,2036-02-23,CALLABLE,Sr Unsecured,1250.000,US,USD,ACTV,20.000000,11.876797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,DIS,Corp,BBG00N3BD9G0,USU25497AR66,US91282CKG59,DIS 7 1/8 04/08/28,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.125,FIXED,...,2019-03-20,2028-04-08,AT MATURITY,Sr Unsecured,10.285,US,USD,ACTV,9.054073,4.000000
820,DIS,Corp,BBG00N3BZ921,USU25497AW51,US91282CJZ59,DIS 8.45 08/01/34,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.450,FIXED,...,2019-03-20,2034-08-01,AT MATURITY,Sr Unsecured,1.000,US,USD,ACTV,15.367556,10.313484
821,DIS,Corp,BBG00N55NLL7,USU25497BM60,US912810TV08,DIS 7.9 12/01/2095,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.900,FIXED,...,2019-03-20,2095-12-01,AT MATURITY,Sr Unsecured,0.502,US,USD,ACTV,76.700890,71.646817
822,DIS,Corp,BBG00N55NPV7,USU25497BN44,US912810TV08,DIS 8 1/4 10/17/2096,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.250,FIXED,...,2019-03-20,2096-10-17,AT MATURITY,Sr Unsecured,0.158,US,USD,ACTV,77.579740,72.525667


In [53]:
bond_symbology= bond_symbology[bond_symbology['cpn_type'] == 'FIXED']
bond_symbology

,ticker,class,figi,isin,und_bench_isin,security,name,type,coupon,cpn_type,...,acc_first,maturity,mty_typ,rank,amt_out,country,currency,status,term,TTM
0,AAPL,Corp,BBG004HST0K7,US037833AL42,US912810TZ12,AAPL 3.85 05/04/43,APPLE INC,GLOBAL,3.850,FIXED,...,2013-05-03,2043-05-04,AT MATURITY,Sr Unsecured,3000.000,US,USD,ACTV,30.001369,19.069131
1,AAPL,Corp,BBG006F8VWJ7,US037833AT77,US912810TZ12,AAPL 4.45 05/06/44,APPLE INC,GLOBAL,4.450,FIXED,...,2014-05-06,2044-05-06,AT MATURITY,Sr Unsecured,1000.000,US,USD,ACTV,30.001369,20.076660
2,AAPL,Corp,BBG0081TNL50,US037833BA77,US912810TZ12,AAPL 3.45 02/09/45,APPLE INC,GLOBAL,3.450,FIXED,...,2015-02-09,2045-02-09,AT MATURITY,Sr Unsecured,2000.000,US,USD,ACTV,30.001369,20.840520
3,AAPL,Corp,BBG008N1BQC1,US037833BH21,US912810TZ12,AAPL 4 3/8 05/13/45,APPLE INC,GLOBAL,4.375,FIXED,...,2015-05-13,2045-05-13,AT MATURITY,Sr Unsecured,2000.000,US,USD,ACTV,30.001369,21.095140
4,AAPL,Corp,BBG00C7QBCQ1,US037833BW97,US91282CJZ59,AAPL 4 1/2 02/23/36,APPLE INC,GLOBAL,4.500,FIXED,...,2016-02-23,2036-02-23,CALLABLE,Sr Unsecured,1250.000,US,USD,ACTV,20.000000,11.876797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,DIS,Corp,BBG00N3BD9G0,USU25497AR66,US91282CKG59,DIS 7 1/8 04/08/28,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.125,FIXED,...,2019-03-20,2028-04-08,AT MATURITY,Sr Unsecured,10.285,US,USD,ACTV,9.054073,4.000000
820,DIS,Corp,BBG00N3BZ921,USU25497AW51,US91282CJZ59,DIS 8.45 08/01/34,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.450,FIXED,...,2019-03-20,2034-08-01,AT MATURITY,Sr Unsecured,1.000,US,USD,ACTV,15.367556,10.313484
821,DIS,Corp,BBG00N55NLL7,USU25497BM60,US912810TV08,DIS 7.9 12/01/2095,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.900,FIXED,...,2019-03-20,2095-12-01,AT MATURITY,Sr Unsecured,0.502,US,USD,ACTV,76.700890,71.646817
822,DIS,Corp,BBG00N55NPV7,USU25497BN44,US912810TV08,DIS 8 1/4 10/17/2096,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.250,FIXED,...,2019-03-20,2096-10-17,AT MATURITY,Sr Unsecured,0.158,US,USD,ACTV,77.579740,72.525667


## b. Add function to construct generic fixed rate cashflow schedules from symbology data

Use one row of the symbology dataframe as input  to the function. Use the helper function to convert a date string to a QuantLib date object.

In [54]:
def get_ql_date(date) -> ql.Date:
    """
    convert dt.date to ql.Date
    """
    if isinstance(date, dt.date):
        return ql.Date(date.day, date.month, date.year)
    elif isinstance(date, str):
        date = dt.datetime.strptime(date, "%Y-%m-%d").date()
        return ql.Date(date.day, date.month, date.year)
    else:
        raise ValueError(f"to_qldate, {type(date)}, {date}")

In [55]:
def create_schedule_from_symbology(details: dict):
    '''Create a QuantLib cashflow schedule from symbology details dictionary (usually one row of the symbology dataframe)
    '''
    # Create maturity from details['maturity']
    maturity = get_ql_date(details['maturity'])    # use date helper function and update code !!!

    # Create acc_first from details['acc_first']
    acc_first = get_ql_date(details['acc_first'])  # use date helper function and update code !!!
    
    # Create calendar for Corp and Govt asset classes
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
    
    # define period from details['cpn_freq'] ... can be hard-coded to 2 = semi-annual frequency
    period = ql.Period(ql.Semiannual)
    
    # business_day_convention
    business_day_convention = ql.Unadjusted
    
    # termination_date_convention
    termination_date_convention = ql.Unadjusted
    
    # date_generation
    date_generation=ql.DateGeneration.Backward
    
    # Create schedule using ql.MakeSchedule interface (with keyword arguments)
    schedule = ql.MakeSchedule(effectiveDate=acc_first,  # this may not be the same as the bond's start date
                            terminationDate=maturity,
                            tenor=period,
                            calendar=calendar,
                            convention=business_day_convention,
                            terminalDateConvention=termination_date_convention,
                            rule=date_generation,
                            endOfMonth=True,
                            firstDate=None,
                            nextToLastDate=None)
    return schedule

## c. Add function to construct generic fixed rate bond objects from symbology data

Use one row of the symbology dataframe as input to the function. Use create_schedule_from_symbology() internally to create the cashflow schedule.

In [56]:
def create_bond_from_symbology(details: dict):
    '''Create a US fixed rate bond object from symbology details dictionary (usually one row of the symbology dataframe)
    '''
    
     # Create day_count from details['dcc']
     # For US Treasuries use ql.ActualActual(ql.ActualActual.ISMA)
     # For US Corporate bonds use ql.Thirty360(ql.Thirty360.USA)
    if details['dcc'] == 'ACT/ACT':
        day_count = ql.ActualActual(ql.ActualActual.ISMA)
    else:
        day_count = ql.Thirty360(ql.Thirty360.USA)   # update code depending on dcc !!!
        
    # Create day_count from details['start_date']    
    issue_date = get_ql_date(details['start_date'])        # use date helper function and update code !!!
    
    # Create days_settle from details['days_settle']
    days_settle = int(details['days_settle'])        # update code depending on days_settle !!!

    # Create days_settle from details['coupon']
    coupon = float(details['coupon']/100)      # update code depending on coupon !!!

    # Create cashflow schedule
    schedule = create_schedule_from_symbology(details)
    
    face_value = 100
    redemption = 100
    
    payment_convention = ql.Unadjusted
        
    # Create fixed rate bond object
    fixed_rate_bond = ql.FixedRateBond(
        days_settle,
        face_value,
        schedule,
        [coupon],
        day_count,
        payment_convention,
        redemption,
        issue_date)        

    return fixed_rate_bond
create_bond_from_symbology(bond_symbology.iloc[0].to_dict())

<QuantLib.QuantLib.FixedRateBond; proxy of <Swig Object of type 'ext::shared_ptr< FixedRateBond > *' at 0x158881aa0> >

## d. Add function that returns a dataframe with (future) cash flows details for a bond object

Use the "Investigate Bond Cashflows" section in the Quantlib introductory notebook as a template.

The results dataframe should contain following columns:


| CashFlowDate | CashFlowAmount | CashFlowYearFrac |
|----------|-------|-------------|

Pick one government and one corporate bond from symbology, create the bond objects and display the future cashflows.

In [57]:
def get_bond_cashflows(bond: ql.FixedRateBond, calc_date=ql.Date) -> pd.DataFrame:
    '''Returns all future cashflows as of calc_date, i.e. with payment dates > calc_date.
    '''    
    day_count = bond.dayCounter()
    x = [(cf.date(), cf.amount(), day_count.yearFraction(calc_date,cf.date())) for cf in bond.cashflows()]
    cf_date_fixed, cf_amount, cf_yearfrac = zip(*x)
    cashflows_df = pd.DataFrame(data={'CashFlowDate': cf_date_fixed, 'CashFlowAmount': cf_amount, 'cf_yearfrac': cf_yearfrac})
    
    return cashflows_df

In [58]:
corp_bond_details = bond_symbology[bond_symbology['class']=='Govt'].reset_index(drop=True).loc[0,:]
corp_bond = create_bond_from_symbology(corp_bond_details)
get_bond_cashflows(corp_bond,calc_date)

,CashFlowDate,CashFlowAmount,cf_yearfrac
0,"November 15th, 1994",3.75,-29.416667
1,"May 15th, 1995",3.75,-28.916667
2,"November 15th, 1995",3.75,-28.416667
3,"May 15th, 1996",3.75,-27.916667
4,"November 15th, 1996",3.75,-27.416667
...,...,...,...
57,"May 15th, 2023",3.75,-0.916667
58,"November 15th, 2023",3.75,-0.416667
59,"May 15th, 2024",3.75,0.083333
60,"November 15th, 2024",3.75,0.583333


# Problem 2: US Treasury yield curve calibration (On-The-Runs)

## a. Create the on-the-run US treasury bond objects

Restrict the symbology + market data dataframe to "on-the-run"/OTR US treasury notes/bonds only and create the treasury bond objects.

Extend the treasuries symbology dataframe with the following market data columns (code from Homework 1):


| date | bidPrice | askPrice | midPrice | bidYield | askYield | midYield | term | TTM |
|----------|-------|-------------|-----|----------|---------|---------|---------|---------|

Plot a graph/scatter plot of on-the-run treasury mid yields by TTM.

In [59]:

maturity_filter = govt_on_the_run['ticker'].str.contains('GT2 |GT3 |GT5 |GT7 |GT10 |GT20 |GT30', regex=True)
govt_on_the_run_treasuries = govt_on_the_run[maturity_filter]    
govt_on_the_run_treasuries

,ticker,date,figi,isin
0,GT10 Govt,2024-04-08,BBG01L8YJFB3,US91282CJZ59
3,GT2 Govt,2024-04-08,BBG01M44ZLG5,US91282CKH33
4,GT20 Govt,2024-04-08,BBG01LK8Y0L1,US912810TZ12
9,GT3 Govt,2024-04-08,BBG01LW4R927,US91282CKE02
10,GT30 Govt,2024-04-08,BBG01L8YJKX8,US912810TX63
11,GT30B Govt,2024-04-08,BBG01JZY4HS2,US912810TV08
12,GT30C Govt,2024-04-08,BBG01HQWRRD9,US912810TT51
15,GT5 Govt,2024-04-08,BBG01M44ZQJ1,US91282CKG59
18,GT7 Govt,2024-04-08,BBG01M44ZS07,US91282CKF76


In [60]:
import pandas as pd
import numpy as np
import plotly.express as px
path = '/Users/rogerlin/Desktop/Temporary/UChicago_FINM_35700_CreditMarkets_Spring2024_Homework_2 2/data/govt_on_the_run.xlsx'
govt_on_the_run = pd.read_excel(path)
govt_on_the_run_filter = ~govt_on_the_run['ticker'].str.contains('B|C', regex=True)
maturity_filter = govt_on_the_run['ticker'].str.contains('GT2|GT3|GT5|GT7|GT10|GT20|GT30', regex=True)
on_the_run_treasuries = govt_on_the_run[govt_on_the_run_filter & maturity_filter]
govt_bonds = bond_symbology[(bond_symbology['ticker'] == 'T') & (bond_symbology['class'] == 'Govt')]
index1 = [np.flatnonzero(govt_bonds['isin'] == isin)[0] for isin in on_the_run_treasuries['isin']]
index2 = [np.flatnonzero(bond_market_prices_eod['isin'] == isin)[0] for isin in on_the_run_treasuries['isin']]
govt_bonds_subset = govt_bonds.iloc[index1]
market_prices_subset = bond_market_prices_eod.iloc[index2]
merged_df = pd.merge(govt_bonds_subset, market_prices_subset, on=['isin', 'class', 'ticker', 'figi'])
merged_df_sorted = merged_df.sort_values(by='TTM')
fig = px.scatter(merged_df_sorted, x='TTM', y='midYield', title='UST Yields by TTM', labels={'TTM': 'Time to Maturity (Years)', 'midYield': 'Mid Yield (%)'})
fig.update_traces(marker=dict(size=8),
                  line=dict(width=2))
fig.show()





## b. Calibrate the on-the-run treasury yield curve (bootstrapping)
The function below shows how to calibrate a smooth yield/discount factor curve from the on-the-run treasury dataframe. 

Calibrate the bid, ask and mid discount factor curves as of 2024-04-08.

Display the calibration results for the mid curve, using get_yield_curve_details_df().

In [61]:
import QuantLib as ql
import pandas as pd

def calibrate_yield_curve(calc_date, treasury_data, price_col):
    """ Calibrates a yield curve from a DataFrame of treasury details including price quotes. """
    ql.Settings.instance().evaluationDate = calc_date
    sorted_data = treasury_data.sort_values(by='maturity')
    day_counter = ql.ActualActual(ql.ActualActual.ISMA)
    bond_helpers = []

    for _, row in sorted_data.iterrows():
        bond = create_bond_from_symbology(row)
        price_quote = ql.QuoteHandle(ql.SimpleQuote(row[price_col]))
        bond_helper = ql.BondHelper(price_quote, bond)
        bond_helpers.append(bond_helper)
    
    yield_curve = ql.PiecewiseLogCubicDiscount(calc_date, bond_helpers, day_counter)
    yield_curve.enableExtrapolation()
    return yield_curve

def get_yield_curve_details(yield_curve, curve_dates=None):
    """ Returns a DataFrame with yield curve details for given curve dates. """
    if curve_dates is None:
        curve_dates = yield_curve.dates()
    
    curve_ql_dates = [get_ql_date(d) for d in curve_dates]
    discounts = [round(yield_curve.discount(d), 3) for d in curve_ql_dates]
    yearfracs = [round(yield_curve.timeFromReference(d), 3) for d in curve_ql_dates]
    zero_rates = [round(yield_curve.zeroRate(d, yield_curve.dayCounter(), ql.Compounded).rate() * 100, 3) for d in curve_ql_dates]
    
    return pd.DataFrame({
        'Date': curve_dates,
        'YearFrac': yearfracs,
        'DiscountFactor': discounts,
        'ZeroRate': zero_rates
    })


In [62]:
def calibrate_yield_curve_from_frame(
        calc_date: ql.Date,
        treasury_details: pd.DataFrame,
        price_quote_column: str):
    '''Create a calibrated yield curve from a details dataframe which includes bid/ask/mid price quotes.
    '''
    ql.Settings.instance().evaluationDate = calc_date

    # Sort dataframe by maturity
    sorted_details_frame = treasury_details.sort_values(by='maturity')    
    
    # For US Treasuries use ql.ActualActual(ql.ActualActual.ISMA)
    day_count = ql.ActualActual(ql.ActualActual.ISMA)

    bond_helpers = []
    
    for index, row in sorted_details_frame.iterrows():
        bond_object = create_bond_from_symbology(row)

        tsy_clean_price_quote = row[price_quote_column]
        tsy_clean_price_handle = ql.QuoteHandle(ql.SimpleQuote(tsy_clean_price_quote))
        
        bond_helper = ql.BondHelper(tsy_clean_price_handle, bond_object)
        bond_helpers.append(bond_helper)
        
    yield_curve = ql.PiecewiseLogCubicDiscount(calc_date, bond_helpers, day_count)
    # yield_curve = ql.PiecewiseFlatForward(calc_date, bond_helpers, day_count)
    
    yield_curve.enableExtrapolation()
    return yield_curve


def get_yield_curve_details_df(yield_curve, curve_dates=None):
    
    if(curve_dates == None):
        curve_dates = yield_curve.dates()

    curve_ql_dates = [get_ql_date(d) for d in curve_dates]
    discounts = [round(yield_curve.discount(d), 3) for d in curve_ql_dates]
    yearfracs = [round(yield_curve.timeFromReference(d), 3) for d in curve_ql_dates]
    zeroRates = [round(yield_curve.zeroRate(d, yield_curve.dayCounter(), ql.Compounded).rate() * 100, 3) for d in curve_ql_dates]

    yield_curve_details_df = pd.DataFrame(data={'Date': curve_dates,
                             'YearFrac': yearfracs,
                             'DiscountFactor': discounts,
                             'ZeroRate': zeroRates})                             
    return yield_curve_details_df

In [65]:
#bidPrice
curve_dates = [(date + pd.DateOffset(months=6 * n)).date() for n in range(61)]
calibrated_curve_details_dic = {}
merged_df_a_sorted.index = pd.Series([0,1,2,3,4,5,6])
calibrated_yield_curve = calibrate_yield_curve_from_frame(calc_date, merged_df_a_sorted, 'bidPrice')
calibrated_curve_details_bid = get_yield_curve_details_df(calibrated_yield_curve, curve_dates)
display(calibrated_curve_details_bid)

,Date,YearFrac,DiscountFactor,ZeroRate
0,2024-04-08,0.0,1.000,4.886
1,2024-10-08,0.5,0.976,4.880
2,2025-04-08,1.0,0.954,4.863
3,2025-10-08,1.5,0.932,4.833
4,2026-04-08,2.0,0.911,4.792
...,...,...,...,...
56,2052-04-08,28.0,0.283,4.610
57,2052-10-08,28.5,0.278,4.595
58,2053-04-08,29.0,0.273,4.581
59,2053-10-08,29.5,0.268,4.567


In [66]:
#askPrice
curve_dates = [(date + pd.DateOffset(months=6 * n)).date() for n in range(61)]
calibrated_curve_details_dic = {}
merged_df_a_sorted.index = pd.Series([0,1,2,3,4,5,6])
calibrated_yield_curve = calibrate_yield_curve_from_frame(calc_date, merged_df_a_sorted, 'askPrice')
calibrated_curve_details_ask = get_yield_curve_details_df(calibrated_yield_curve, curve_dates)
display(calibrated_curve_details_ask)

,Date,YearFrac,DiscountFactor,ZeroRate
0,2024-04-08,0.0,1.000,4.880
1,2024-10-08,0.5,0.976,4.875
2,2025-04-08,1.0,0.954,4.857
3,2025-10-08,1.5,0.932,4.828
4,2026-04-08,2.0,0.911,4.788
...,...,...,...,...
56,2052-04-08,28.0,0.283,4.609
57,2052-10-08,28.5,0.278,4.595
58,2053-04-08,29.0,0.273,4.581
59,2053-10-08,29.5,0.268,4.567


In [67]:
#midPrice
curve_dates = [(date + pd.DateOffset(months=6 * n)).date() for n in range(61)]
calibrated_curve_details_dic = {}
merged_df_a_sorted.index = pd.Series([0,1,2,3,4,5,6])
calibrated_yield_curve = calibrate_yield_curve_from_frame(calc_date, merged_df_a_sorted, 'midPrice')
calibrated_curve_details_mid = get_yield_curve_details_df(calibrated_yield_curve, curve_dates)
display(calibrated_curve_details_mid)

,Date,YearFrac,DiscountFactor,ZeroRate
0,2024-04-08,0.0,1.000,4.883
1,2024-10-08,0.5,0.976,4.877
2,2025-04-08,1.0,0.954,4.860
3,2025-10-08,1.5,0.932,4.831
4,2026-04-08,2.0,0.911,4.790
...,...,...,...,...
56,2052-04-08,28.0,0.283,4.610
57,2052-10-08,28.5,0.278,4.595
58,2053-04-08,29.0,0.273,4.581
59,2053-10-08,29.5,0.268,4.567


## c. Plot the calibrated US Treasury yield (zero rate) curves

Create a graph/scatter plot of the newly computed mid yields by maturity.

In [71]:
def plot_ust_yields(df):
    """
    Generate an interactive plot of UST yields by TTM using Plotly.

    Args:
    df (pd.DataFrame): DataFrame containing the yield curve details with 'YearFrac' and 'ZeroRate'.
    """
    fig = px.scatter(df, x='YearFrac', y='ZeroRate',
                     labels={'YearFrac': 'Time to Maturity (Years)', 'ZeroRate': 'Zero Rate (%)'},
                     title='UST Yields by TTM')
    fig.update_layout(xaxis_title='Time to Maturity (Years)',
                      yaxis_title='Zero Rate (%)',
                      title={'text': 'UST Yields by TTM', 'x':0.5, 'xanchor': 'center'})
    fig.show()
plot_ust_yields(calibrated_curve_details_mid)


## d. Plot calibrated discount factors

Plot the discount factor curve up to the 30 years point, using a 6 months discretization grid.

In [72]:
def plot_ust_discount_factors(df):
    """
    Generate an interactive plot of UST discount factors by TTM using Plotly.

    Args:
    df (pd.DataFrame): DataFrame containing the yield curve details with 'YearFrac' and 'DiscountFactor'.
    """
    fig = px.scatter(df, x='YearFrac', y='DiscountFactor',
                     labels={'YearFrac': 'Time to Maturity (Years)', 'DiscountFactor': 'Discount Factor'},
                     title='UST Discount Factor by TTM')
    fig.update_layout(xaxis_title='Time to Maturity (Years)',
                      yaxis_title='Discount Factor',
                      title={'text': 'UST Discount Factor by TTM', 'x':0.5, 'xanchor': 'center'})
    fig.show()
plot_ust_discount_factors(calibrated_curve_details_mid)


# Problem 3: Pricing and risk metrics for US Treasury bonds

## a. US Treasury pricing on the calibrated discount factor curve

Follow Section 5. "Bond Present Value Calculation (no credit risk)" in the QuantLib Basic notebook to re-price the US on-the-run treasuries using the calibrated discount factor curve. 

You will need to switch the bond_engine to use the new on-the-run treasury yield curve:
bond_engine = ql.DiscountingBondEngine(tsy_yield_curve_mid)

Extend the dataframe with the following computed columns for clean mid prices:


| calc_mid_price |
|---------------|


To validate the calibration, compare the calculated clean mid prices to the original market mid prices.

In [ ]:
mid_yield_handle = ql.YieldTermStructureHandle(calibratied_yield_curve)
bond_engine = ql.DiscountingBondEngine(mid_yield_handle)
calibrated_price = pd.DataFrame({'isin': merged_df_a_sorted['isin'], 'calc_mid_price':np.nan})
for index, row in merged_df_a_sorted.iterrows():
    bond_object = create_bond_from_symbology(row)
    bond_object.setPricingEngine(bond_engine)
    calibrated_price.loc[index, 'calc_mid_price'] = bond_object.cleanPrice()
merged_df_b_sorted = pd.merge(merged_df_a_sorted,calibrated_price,on='isin')
merged_df_b_sorted[['isin','calc_mid_price']]

,isin,calc_mid_price
0,US91282CKH33,99.45700
1,US91282CKE02,99.03520
2,US91282CKG59,98.64450
3,US91282CKF76,98.16410
4,US91282CJZ59,96.65625
5,US912810TZ12,97.96875
6,US912810TX63,95.11720


## b. Compute analytical DV01, Duration and Convexity for US on-the-run treasuries (using flat yield)

Compute analytical DV01, Duration and Convexity metrics, as described in Section 7. "Analytics Duration, Convexity and Z-Spread (flat yield model)" in the QuantLib Basic notebook.

Remember that DV01 = Dirty_Price * Duration. 

Extend the dataframe with the following calculated risk metrics:


| dv01 | duration | convexity |
|-------|-------|-------------|


In [ ]:
flat_yield = 0.05
days = ql.Actual360()
compound = ql.Compounded
cpn_fre = ql.Semiannual
rate = ql.InterestRate(flat_yield, days, compound, cpn_fre)

data3_b = pd.DataFrame({
    'isin': merged_df_b_sorted['isin'],
    'dv01': np.nan,
    'duration': np.nan,
    'convexity': np.nan,
})

for index, row in merged_df_b_sorted.iterrows():
    bond_object = create_bond_from_symbology(row)
    bond_object.setPricingEngine(bond_engine)
    duration = ql.BondFunctions.duration(bond_object, rate)
    convexity = ql.BondFunctions.convexity(bond_object, rate)
    data3_b.loc[index,'duration'] = duration
    data3_b.loc[index,'convexity'] = convexity
    data3_b.loc[index,'dv01'] = duration * bond_object.dirtyPrice() / 100

merged_df_c_sorted = pd.merge(merged_df_b_sorted,data3_b, on='isin')
merged_df_c_sorted[['isin','dv01','duration','convexity']]

,isin,dv01,duration,convexity
0,US91282CKH33,1.881078,1.889246,4.561690
1,US91282CKE02,2.729206,2.747783,9.145758
2,US91282CKG59,4.430112,4.486374,23.522451
3,US91282CKF76,5.928043,6.032678,42.557796
4,US91282CJZ59,7.799071,8.019639,76.814375
5,US912810TZ12,12.657386,12.832377,220.109919
6,US912810TX63,15.311773,15.991792,375.397348


## c. Compute scenario DV01, Duration and Convexity for US on-the-run treasuries (using calibrated yield curve)

Compute the scenario DV01, Duration and Convexity metrics using +/-1bp interest rate shocks, as described in Section 6. "Market Data Scenarios" in the QuantLib Basic notebook.

Remember that DV01 = Dirty_Price * Duration.

Extend the dataframe with the following scenario sensitivities metrics:

| scen_dv01 | scen_duration | scen_convexity |
|-------|-------|-------------|



In [ ]:
scenario_df = pd.DataFrame({
    'isin': merged_df_c_sorted['isin'],
    'scen_dv01': np.nan,
    'scen_duration': np.nan,
    'scen_convexity': np.nan,
})

for index, row in merged_df_c_sorted.iterrows():
    bond_object = create_bond_from_symbology(row)
    interest_rate = ql.SimpleQuote(0.0)
    yield_curve = ql.ZeroSpreadedTermStructure(mid_yield_handle, ql.QuoteHandle(interest_rate))
    bond_engine = ql.DiscountingBondEngine(ql.YieldTermStructureHandle(yield_curve))
    bond_object.setPricingEngine(bond_engine)
    price_base = bond_object.cleanPrice()
    interest_rate.setValue(0.0001)
    big_1bp = bond_object.cleanPrice()
    interest_rate.setValue(-0.0001)
    small_1bp = bond_object.cleanPrice()
    interest_rate.setValue(0)
    dv01 = (small_1bp - price_base) * 10000 / 100
    duration = dv01 / bond_object.dirtyPrice() * 100
    gamma_1bp = (small_1bp - 2*price_base + big_1bp) * 100000000 / 100
    convexity = gamma_1bp / bond_object.dirtyPrice() * 100
    scenario_df.loc[index, 'scen_duration'] = duration
    scenario_df.loc[index, 'scen_convexity'] = gamma_1bp
    scenario_df.loc[index, 'scen_dv01'] = dv01

merged_df_d_sorted = pd.merge(merged_df_c_sorted,scenario_df, on='isin')
merged_df_d_sorted[['isin','scen_duration','scen_convexity','scen_dv01']]

,isin,scen_duration,scen_convexity,scen_dv01
0,US91282CKH33,1.932266,3.788688,1.923912
1,US91282CKE02,2.761765,7.831008,2.743093
2,US91282CKG59,4.565894,21.766400,4.508634
3,US91282CKF76,6.144130,40.222328,6.037563
4,US91282CJZ59,8.141551,72.676821,7.917630
5,US912810TZ12,13.093005,217.588317,12.914460
6,US912810TX63,16.761024,378.958839,16.048294


# Problem 4: Pricing and risk metrics for corporate bonds

## a. Create the fixed-rate corporate bond objects

Restrict the symbology dataframe to fixed rate corporate bonds only and create the corporate bond objects.

In [ ]:
corp_bond_symbology=bond_symbology[bond_symbology['class'] == 'Corp']
corp_bond_symbology

,ticker,class,figi,isin,und_bench_isin,security,name,type,coupon,cpn_type,...,acc_first,maturity,mty_typ,rank,amt_out,country,currency,status,term,TTM
0,AAPL,Corp,BBG004HST0K7,US037833AL42,US912810TZ12,AAPL 3.85 05/04/43,APPLE INC,GLOBAL,3.850,FIXED,...,2013-05-03,2043-05-04,AT MATURITY,Sr Unsecured,3000.000,US,USD,ACTV,30.001369,19.069131
1,AAPL,Corp,BBG006F8VWJ7,US037833AT77,US912810TZ12,AAPL 4.45 05/06/44,APPLE INC,GLOBAL,4.450,FIXED,...,2014-05-06,2044-05-06,AT MATURITY,Sr Unsecured,1000.000,US,USD,ACTV,30.001369,20.076660
2,AAPL,Corp,BBG0081TNL50,US037833BA77,US912810TZ12,AAPL 3.45 02/09/45,APPLE INC,GLOBAL,3.450,FIXED,...,2015-02-09,2045-02-09,AT MATURITY,Sr Unsecured,2000.000,US,USD,ACTV,30.001369,20.840520
3,AAPL,Corp,BBG008N1BQC1,US037833BH21,US912810TZ12,AAPL 4 3/8 05/13/45,APPLE INC,GLOBAL,4.375,FIXED,...,2015-05-13,2045-05-13,AT MATURITY,Sr Unsecured,2000.000,US,USD,ACTV,30.001369,21.095140
4,AAPL,Corp,BBG00C7QBCQ1,US037833BW97,US91282CJZ59,AAPL 4 1/2 02/23/36,APPLE INC,GLOBAL,4.500,FIXED,...,2016-02-23,2036-02-23,CALLABLE,Sr Unsecured,1250.000,US,USD,ACTV,20.000000,11.876797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,DIS,Corp,BBG00N3BD9G0,USU25497AR66,US91282CKG59,DIS 7 1/8 04/08/28,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.125,FIXED,...,2019-03-20,2028-04-08,AT MATURITY,Sr Unsecured,10.285,US,USD,ACTV,9.054073,4.000000
820,DIS,Corp,BBG00N3BZ921,USU25497AW51,US91282CJZ59,DIS 8.45 08/01/34,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.450,FIXED,...,2019-03-20,2034-08-01,AT MATURITY,Sr Unsecured,1.000,US,USD,ACTV,15.367556,10.313484
821,DIS,Corp,BBG00N55NLL7,USU25497BM60,US912810TV08,DIS 7.9 12/01/2095,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.900,FIXED,...,2019-03-20,2095-12-01,AT MATURITY,Sr Unsecured,0.502,US,USD,ACTV,76.700890,71.646817
822,DIS,Corp,BBG00N55NPV7,USU25497BN44,US912810TV08,DIS 8 1/4 10/17/2096,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.250,FIXED,...,2019-03-20,2096-10-17,AT MATURITY,Sr Unsecured,0.158,US,USD,ACTV,77.579740,72.525667


In [ ]:
corp_bond_objects = [create_bond_from_symbology(corp_bond_symbology.loc[i].to_dict()) for i in corp_bond_symbology.index]

## b. Compute analytical Yields and Z-Spreads

Compute analytical Yields and Z-Spreads metrics, as described in Section 7. "Analytical Duration, Convexity and Z-Spread (flat yield model)" in the QuantLib Basic notebook.

Extend the dataframe with the following calculated risk metrics:


| calc_yield | calc_zspread |
|-------|-------------|


In [ ]:
corp_bond = pd.merge(corp_bond_symbology,bond_market_prices_eod[['isin','date','bidPrice','askPrice','bidYield','askYield', 'midPrice', 'midYield']], on='isin', how='left')
days = ql.Actual360()
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
compound = ql.Compounded
flat_rate = ql.SimpleQuote(0.049)
flat_rate_handle = ql.QuoteHandle(flat_rate)
flat_yield = ql.FlatForward(calc_date, flat_rate_handle, days, compound)
flat_yield_handle = ql.YieldTermStructureHandle(flat_yield)

for index, row in corp_bond.iterrows():
    bond_object = create_bond_from_symbology(row)
    bond_object.setPricingEngine(bond_engine)
    settle_date = bond_object.settlementDate(calc_date)
    day_counter = bond_object.dayCounter()
    implied_yield = bond_object.bondYield(row['midPrice'], day_counter, compound, cpn_fre, settle_date) * 100
    corp_bond.loc[index, 'implied_yield'] = implied_yield
    bond_zspread = ql.BondFunctions.zSpread(bond_object, row['midPrice'], flat_yield, days, compound, cpn_fre, settle_date)
    corp_bond.loc[index, 'calc_zspread'] = bond_zspread
corp_bond

,ticker,class,figi,isin,und_bench_isin,security,name,type,coupon,cpn_type,...,TTM,date,bidPrice,askPrice,bidYield,askYield,midPrice,midYield,implied_yield,calc_zspread
0,AAPL,Corp,BBG004HST0K7,US037833AL42,US912810TZ12,AAPL 3.85 05/04/43,APPLE INC,GLOBAL,3.850,FIXED,...,19.069131,2024-04-08,84.578,85.153,5.127,5.074,84.8655,5.1005,5.100419,0.001853
1,AAPL,Corp,BBG006F8VWJ7,US037833AT77,US912810TZ12,AAPL 4.45 05/06/44,APPLE INC,GLOBAL,4.450,FIXED,...,20.076660,2024-04-08,92.578,93.293,5.042,4.982,92.9355,5.0120,5.011954,0.000978
2,AAPL,Corp,BBG0081TNL50,US037833BA77,US912810TZ12,AAPL 3.45 02/09/45,APPLE INC,GLOBAL,3.450,FIXED,...,20.840520,2024-04-08,78.406,78.952,5.152,5.102,78.6790,5.1270,5.127133,0.002101
3,AAPL,Corp,BBG008N1BQC1,US037833BH21,US912810TZ12,AAPL 4 3/8 05/13/45,APPLE INC,GLOBAL,4.375,FIXED,...,21.095140,2024-04-08,90.009,90.602,5.157,5.108,90.3055,5.1325,5.132464,0.002168
4,AAPL,Corp,BBG00C7QBCQ1,US037833BW97,US91282CJZ59,AAPL 4 1/2 02/23/36,APPLE INC,GLOBAL,4.500,FIXED,...,11.876797,2024-04-08,97.413,97.894,4.788,4.733,97.6535,4.7605,4.760493,-0.001509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,DIS,Corp,BBG00N3BD9G0,USU25497AR66,US91282CKG59,DIS 7 1/8 04/08/28,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.125,FIXED,...,4.000000,2024-04-08,106.909,108.048,5.188,4.883,107.4785,5.0355,5.034684,0.001201
433,DIS,Corp,BBG00N3BZ921,USU25497AW51,US91282CJZ59,DIS 8.45 08/01/34,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.450,FIXED,...,10.313484,2024-04-08,123.831,124.870,5.403,5.288,124.3505,5.3455,5.345199,0.004253
434,DIS,Corp,BBG00N55NLL7,USU25497BM60,US912810TV08,DIS 7.9 12/01/2095,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.900,FIXED,...,71.646817,2024-04-08,129.908,131.277,6.061,5.996,130.5925,6.0285,6.028462,0.010991
435,DIS,Corp,BBG00N55NPV7,USU25497BN44,US912810TV08,DIS 8 1/4 10/17/2096,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.250,FIXED,...,72.525667,2024-04-08,133.747,135.288,6.149,6.077,134.5175,6.1130,6.112755,0.011825


## c. Validate Z-Spread computation for a few fixed rate corporate bonds

Pick 3 corporate bonds (at your discretion) and use function below to re-price them using the calibrated flat z-spread. Follow the example in Section 7. "Analytical Duration, Convexity and Z-Spread (flat yield model)".

Validate that you match the original market prices, which were used as input to the z-Spread function.


In [ ]:
def calc_clean_price_with_zspread(fixed_rate_bond, yield_curve_handle, zspread):
    zspread_quote = ql.SimpleQuote(zspread)
    zspread_quote_handle = ql.QuoteHandle(zspread_quote)
    yield_curve_bumped = ql.ZeroSpreadedTermStructure(yield_curve_handle, zspread_quote_handle, ql.Compounded, ql.Semiannual)
    yield_curve_bumped_handle = ql.YieldTermStructureHandle(yield_curve_bumped)
    
    # Set Valuation engine
    bond_engine = ql.DiscountingBondEngine(yield_curve_bumped_handle)
    fixed_rate_bond.setPricingEngine(bond_engine)
    bond_clean_price = fixed_rate_bond.cleanPrice()
    return bond_clean_price


In [ ]:
corp_bond_c = corp_bond.groupby('ticker')
corp_bond_c_new = corp_bond_c.first().reset_index()[0:3]
for index, row in corp_bond_c_new.iterrows():
    corp_bond_object = create_bond_from_symbology(row)
    midprice = row['midPrice']
    bond_zspread_price = calc_clean_price_with_zspread(corp_bond_object, flat_yield_handle, row['calc_zspread'])
    corp_bond_c_new.loc[index, 'market_mid_price'] = midprice
    corp_bond_c_new.loc[index, 'calc_zspread_price'] = bond_zspread_price
corp_bond_c_new

,ticker,class,figi,isin,und_bench_isin,security,name,type,coupon,cpn_type,...,bidPrice,askPrice,bidYield,askYield,midPrice,midYield,implied_yield,calc_zspread,market_mid_price,calc_zspread_price
0,AAPL,Corp,BBG004HST0K7,US037833AL42,US912810TZ12,AAPL 3.85 05/04/43,APPLE INC,GLOBAL,3.850,FIXED,...,84.578,85.153,5.127,5.074,84.8655,5.1005,5.100419,0.001853,84.8655,84.8655
1,DIS,Corp,BBG00QNKGJP6,US254687DK97,US91282CKH33,DIS 3 3/8 11/15/26,WALT DISNEY COMPANY/THE,GLOBAL,3.375,FIXED,...,95.894,96.146,5.081,4.974,96.0200,5.0275,5.027415,0.001111,96.0200,96.0200
2,F,Corp,BBG00R9PG6L1,US345397A456,US91282CKE02,F 4.271 01/09/27,FORD MOTOR CREDIT CO LLC,GLOBAL,4.271,FIXED,...,95.722,95.982,5.980,5.873,95.8520,5.9265,5.926229,0.009951,95.8520,95.8520


## d. Compute Duration and Convexity for fixed rate corporate bonds (using flat yield)

Compute analytical Duration and Convexity metrics, as described in Section 7. "Analytical Duration, Convexity and Z-Spread (flat yield model)" in the QuantLib Basic notebook.

Extend the dataframe with the following calculated risk metrics:


| calc_duration | calc_convexity |
|-------|-------------|

Display the head of the dataframe.

In [ ]:
flat_yield = 5.1
flat_yield_rate = ql.InterestRate(flat_yield/100, days, compound, cpn_fre)

corp_bond_d = pd.DataFrame({
    'isin': corp_bond['isin'],
    'dv01': np.nan,
    'duration': np.nan,
    'convexity': np.nan,
})
for index, row in corp_bond.iterrows():
    bond_object = create_bond_from_symbology(row)
    bond_object.setPricingEngine(bond_engine)
    duration = ql.BondFunctions.duration(bond_object, flat_yield_rate)
    convexity = ql.BondFunctions.convexity(bond_object, flat_yield_rate)
    corp_bond_d.loc[index, 'duration'] = duration
    corp_bond_d.loc[index, 'convexity'] = convexity
    corp_bond_d.loc[index, 'dv01'] = duration * bond_object.dirtyPrice() / 100

corp_bond_d_new = pd.merge(corp_bond,corp_bond_d, on='isin')
corp_bond_d_new

,ticker,class,figi,isin,und_bench_isin,security,name,type,coupon,cpn_type,...,askPrice,bidYield,askYield,midPrice,midYield,implied_yield,calc_zspread,dv01,duration,convexity
0,AAPL,Corp,BBG004HST0K7,US037833AL42,US912810TZ12,AAPL 3.85 05/04/43,APPLE INC,GLOBAL,3.850,FIXED,...,85.153,5.127,5.074,84.8655,5.1005,5.100419,0.001853,11.622848,12.721344,214.213721
1,AAPL,Corp,BBG006F8VWJ7,US037833AT77,US912810TZ12,AAPL 4.45 05/06/44,APPLE INC,GLOBAL,4.450,FIXED,...,93.293,5.042,4.982,92.9355,5.0120,5.011954,0.000978,12.599341,12.715945,219.881997
2,AAPL,Corp,BBG0081TNL50,US037833BA77,US912810TZ12,AAPL 3.45 02/09/45,APPLE INC,GLOBAL,3.450,FIXED,...,78.952,5.152,5.102,78.6790,5.1270,5.127133,0.002101,11.724114,13.889088,254.013942
3,AAPL,Corp,BBG008N1BQC1,US037833BH21,US912810TZ12,AAPL 4 3/8 05/13/45,APPLE INC,GLOBAL,4.375,FIXED,...,90.602,5.157,5.108,90.3055,5.1325,5.132464,0.002168,12.876346,13.140231,236.687789
4,AAPL,Corp,BBG00C7QBCQ1,US037833BW97,US91282CJZ59,AAPL 4 1/2 02/23/36,APPLE INC,GLOBAL,4.500,FIXED,...,97.894,4.788,4.733,97.6535,4.7605,4.760493,-0.001509,9.124047,9.077989,101.380206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,DIS,Corp,BBG00N3BD9G0,USU25497AR66,US91282CKG59,DIS 7 1/8 04/08/28,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.125,FIXED,...,108.048,5.188,4.883,107.4785,5.0355,5.034684,0.001201,3.860701,3.524189,15.029197
433,DIS,Corp,BBG00N3BZ921,USU25497AW51,US91282CJZ59,DIS 8.45 08/01/34,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.450,FIXED,...,124.870,5.403,5.288,124.3505,5.3455,5.345199,0.004253,9.800816,7.296231,68.805995
434,DIS,Corp,BBG00N55NLL7,USU25497BM60,US912810TV08,DIS 7.9 12/01/2095,WALT DISNEY COMPANY/THE,EURO-DOLLAR,7.900,FIXED,...,131.277,6.061,5.996,130.5925,6.0285,6.028462,0.010991,32.949621,18.304055,626.913885
435,DIS,Corp,BBG00N55NPV7,USU25497BN44,US912810TV08,DIS 8 1/4 10/17/2096,WALT DISNEY COMPANY/THE,EURO-DOLLAR,8.250,FIXED,...,135.288,6.149,6.077,134.5175,6.1130,6.112755,0.011825,34.395477,18.185708,623.647679
